#### In this notebook, we show how to simply grab tweets from Twitter and use sentiment analysis with TextBlob

In [41]:
import tweepy
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer ## French adaptor
import numpy as np
import pandas as pd

In [60]:
## Use user keys & tokens
consumer_key = 
consumer_secret = 

access_token = 
access_token_secret = 

## Authentification
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [61]:
## Prepare a function to simply get a global sentiment
def get_label(analysis, threshold = 0):
    if analysis.sentiment[0] > threshold:
        return 'Positive'
    elif analysis.sentiment[0] == threshold:
        return 'Neutral'
    else:
        return 'Negative'

#### Let's begin the query, with a function that outputs the average sentiment score, the minimum and maximum scores and the related tweets

In [90]:
## Provide starting and final dates
since_date = "2016-10-15"
until_date = "2016-10-25"

## Select tweets containing your query, specifying the number you want and the dates range

def get_tweets(query, count):
    
    public_tweets = api.search(query, count = count, since = since_date, until = until_date)

    polarity = []
    result = []
    for tweet in public_tweets:
        #print(tweet.text)
        analysis = TextBlob(tweet.text, pos_tagger=PatternTagger()
                            , analyzer=PatternAnalyzer())
        polarity.append(analysis.sentiment[0])
        result.append([tweet.text, get_label(analysis), analysis.sentiment[0]])
    return pd.DataFrame(result, columns = ['Tweet', 'Sentiment', 'Figure'])

In [95]:
candidates_list = ['Macron', "Hollande", "Sarkozy", "Mélenchon", "Lepen", "Bayrou", "Duflot", "Copé", "Lesquen"]

In [107]:
for candidate in candidates_list:
    data = get_tweets(candidate, 100)
    print(candidate)
    ratio = len(data[data.Sentiment == 'Positive']) / len(data)
    print(ratio)
    average = data.Figure.mean()
    maxi = data.Figure.max()
    tweet_max = data[data.Figure == maxi].Tweet.tolist()[0]
    mini = data.Figure.min()
    tweet_min = data[data.Figure == mini].Tweet.tolist()[0]
    print("Average score: ", average)
    print("Max score: ", maxi)
    print("Min score: ", mini)
    print("Max tweet: ", tweet_max)
    print("Min tweet: ", tweet_min)
    print('---------------------------------')

Macron
0.41
Average score:  0.05089718749999998
Max score:  1.0
Min score:  -1.0
Max tweet:  Avoir de l'empathie c'est la plus belle marque d'intelligence ce qu'est dénué le #FN #Hollande #Valls #Macron #Sarkozy #cope #Cazeneuve
Min tweet:  Après les cars #Macron, voici les cars #Cazeneuve.
Moi qui prenais #Hollande pour la moitié d'un imbécile, je me tr… https://t.co/rXZdhARWAR
---------------------------------
Hollande
0.28
Average score:  0.013358333333333338
Max score:  0.38
Min score:  -0.38
Max tweet:  #PSG : PSG-OM: Louis et Pierre Sarkozy, frères complices face à Thomas Hollande in love https://t.co/IBtoBRZL27
Min tweet:  RT @chesterfizz222: Celui qui a mis avec notre pays en lambeaux avec  Hollande veut rassembler? Ça devient grotesque à gauche. https://t.co…
---------------------------------
Sarkozy
0.55
Average score:  0.08660999999999996
Max score:  1.0
Min score:  -0.8
Max tweet:  RT @SylvieGaillard: #Sarkozy!!... byebye!! plus jamais @NicolasSarkozy fini!!!! snifff! pour 

#### Now, let's return the top N positive tweets and top N negative tweets

In [121]:
## Provide starting and final dates
since_date = "2016-10-15"
until_date = "2016-10-25"

def top_tweets(query, count, N):
    
    public_tweets = api.search(query, count = count, since = since_date, until = until_date)

    polarity = []
    result = []
    for tweet in public_tweets:
        #print(tweet.text)
        analysis = TextBlob(tweet.text, pos_tagger=PatternTagger()
                            , analyzer=PatternAnalyzer())
        polarity.append(analysis.sentiment[0])
        result.append([tweet.text, get_label(analysis), analysis.sentiment[0]])
    result = pd.DataFrame(result, columns = ['Tweet', 'Sentiment', 'Figure'])
    result = result.sort('Figure', ascending = False)
    top_N = result.Tweet[:N]
    bottom_N = result.Tweet[-N:]
    return top_N.tolist(), bottom_N.tolist()

In [122]:
for candidate in candidates_list:
    positives, negatives = top_tweets(candidate, 100, 10)
    print(candidate)
    print("Postive tweets: ", positives)
    print("Negative tweets: ", negatives)
    print('---------------------------------')

C:\Users\Peter martigny\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Macron
Postive tweets:  ["Avoir de l'empathie c'est la plus belle marque d'intelligence ce qu'est dénué le #FN #Hollande #Valls #Macron #Sarkozy #cope #Cazeneuve", "Excellent Emmanuel Macron sur l'identité de notre pays!! https://t.co/09IKhJ6bRs", "RT @AGMarquet: Purée en première lecture j'avais compris #LaVacheQuiRit 😂😂😂 ! https://t.co/gi8DRZWsWI", 'PtitConDe @EmmanuelMacron https://t.co/lLdx2j98jW va en + leur piquer des électeurs/moutons :-) \n@IsabelleForger @NicolasSarkozy @bayrou', 'RT @Votre_Pere: De nouveaux autocars Macron sont mis en service à #Calais ... https://t.co/TSSKW7otzv', '@souriaalep @russeurope Eh oui on ne veut pas de hollande alors on tt faire sur macron ou Vlalls!! Ils forts nos élites !!', 'RT @jccambadelis: Un rassemblement de Hamon à Macron, ce serait intéressant dans la perspective de la #présidentielle2017 . #primaire #inte…', "RT @LaurencePache: Merci Mr #Vidalies, pour l'ouverture à la concurrence des #TER, pour l'augmentation des péages autoroutiers, po